In [1626]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize

NUM_INPUT = 784  # Number of input neurons
NUM_HIDDEN = 40  # Number of hidden neurons
NUM_OUTPUT = 10  # Number of output neurons
NUM_CHECK = 5  # Number of examples on which to check the gradient

### Helper Functions

In [1664]:
def reluprime(x):
    return 1 * (x > 0)

In [1665]:
def relu(x):
    return np.maximum(x,0)

array([[1, 2],
       [1, 2],
       [1, 3],
       [1, 4],
       [1, 5]])

In [1629]:
def hotten(y):
    new_Y = np.zeros((10,y.shape[0]))
    for i,val in enumerate(y):
        new_Y[val][i] = 1
    return new_Y.T

In [1630]:
def softmax(x):
#     x -= np.max(x,axis=0).reshape(1,-1)
    z = np.exp(x) / np.sum(np.exp(x),axis=0)
    return z.T

In [1673]:
def get_yHat(x,w):
    W1, b1, W2, b2 = unpack(w)
    z1 = np.dot(W1,x.T) + b1[:,None]
    print(z1[0])
    print()
    h = relu(z1)
    print(h[0])
    z2 = np.dot(W2,h) + b2[:,None]
    return softmax(z2)

In [1674]:
get_yHat(trainX[idxs,:],w)

[ 0.03234724 -0.05019588  0.18442815  0.1985927   0.16782607]

[0.03234724 0.         0.18442815 0.1985927  0.16782607]


array([[0.0968067 , 0.09043558, 0.10564901, 0.0948646 , 0.10568138,
        0.09728182, 0.0986594 , 0.10892358, 0.09916343, 0.10253451],
       [0.10131417, 0.08933695, 0.11058904, 0.08544749, 0.10113344,
        0.09804517, 0.108772  , 0.11008004, 0.10259179, 0.0926899 ],
       [0.09293249, 0.08052613, 0.11565522, 0.0778346 , 0.09684647,
        0.11163511, 0.10769651, 0.13863393, 0.09728818, 0.08095137],
       [0.09676678, 0.08578867, 0.11119244, 0.08898187, 0.10471422,
        0.10176497, 0.09868901, 0.11740705, 0.10064938, 0.09404561],
       [0.0851679 , 0.07390499, 0.12312738, 0.08035636, 0.10011125,
        0.10060435, 0.10849624, 0.14168244, 0.09946948, 0.08707962]])

### Given Functions

In [1632]:
# Given a vector w containing all the weights and biased vectors, extract
# and return the individual weights and biases W1, b1, W2, b2.
# This is useful for performing a gradient check with check_grad.
def unpack (w):
    W1 = w[0:NUM_INPUT * NUM_HIDDEN]
    W1 = np.reshape(W1, (NUM_HIDDEN, NUM_INPUT))
    w = np.delete(w, np.arange(NUM_INPUT * NUM_HIDDEN))
    b1 = w[0:NUM_HIDDEN]
    w = np.delete(w, np.arange(NUM_HIDDEN))
    W2 = w[0:NUM_HIDDEN * NUM_OUTPUT]
    W2 = np.reshape(W2, (NUM_OUTPUT, NUM_HIDDEN))
    w = np.delete(w, np.arange(NUM_HIDDEN * NUM_OUTPUT))
    b2 = w[0:NUM_OUTPUT]
    return W1, b1, W2, b2

In [1633]:
# Given individual weights and biases W1, b1, W2, b2, concatenate them and
# return a vector w containing all of them.
# This is useful for performing a gradient check with check_grad.
def pack (W1, b1, W2, b2):
    W = W1.flatten()
    W = np.append(W, b1)
    W = np.append(W, W2.flatten())
    W = np.append(W, b2)
    return W

In [1634]:
# Load the images and labels from a specified dataset (train or test).
def loadData (which):
    images = np.load("fashion_mnist_{}_images.npy".format(which))
    labels = np.load("fashion_mnist_{}_labels.npy".format(which))
    labels = hotten(labels)
    return images, labels

In [1635]:
from sklearn.metrics import log_loss
# Given training images X, associated labels Y, and a vector of combined weights
# and bias terms w, compute and return the cross-entropy (CE) loss. You might
# want to extend this function to return multiple arguments (in which case you
# will also need to modify slightly the gradient check code below).
def fCE (X, Y, w):
    yhat = get_yHat(X, w) #(5,10)
    cost = np.sum((Y*np.log(yhat)))
    cost = cost * (-1/Y.shape[0])
    return cost

In [1636]:
fCE((trainX[idxs, :]),(trainY[idxs]), w)

2.281532786812229

In [1646]:
# Given training images X, associated labels Y, and a vector of combined weights
# and bias terms w, compute and return the gradient of fCE. You might
# want to extend this function to return multiple arguments (in which case you
# will also need to modify slightly the gradient check code below).
def gradCE (X, Y, w):
    W1, b1, W2, b2 = unpack(w)
    yHat = get_yHat(X,w) #(10,5)
    z1 = W1.dot(X.T) + b1[:,None]
    h = reluprime(z1)
    
    gradW2fCE = (yHat - Y).T.dot(h.T) # (10,40)
    gradb2fCE = np.mean(yHat - Y,axis=0).reshape(10,1) #(10,1)
    
    g_Trans = np.multiply(((yHat-Y).dot(W2)),reluprime(z1.T)) #(5,40)
    gradW1fCE = g_Trans.T.dot(X) # (40,785)
    gradb1fCE = np.mean(g_Trans,axis=0).reshape(40,1) # (40,1)

        
    return pack(gradW1fCE,gradb1fCE,gradW2fCE,gradb2fCE)

In [1647]:
gradCE((trainX[idxs, :]),(trainY[idxs]), w)

array([ 0.        ,  0.        ,  0.        , ...,  0.08717712,
       -0.10179899, -0.10632816])

In [1648]:
# Given training and testing datasets and an initial set of weights/biases b,
# train the NN.
def train (trainX, trainY, testX, testY, w):
    pass

## Main

In [1649]:
trainX, trainY = loadData("train")
testX, testY = loadData("test")

trainX = trainX/255.00

# Initialize weights randomly
W1 = 2*(np.random.random(size=(NUM_HIDDEN, NUM_INPUT))/NUM_INPUT**0.5) - 1./NUM_INPUT**0.5
b1 = 0.01 * np.ones(NUM_HIDDEN)
W2 = 2*(np.random.random(size=(NUM_OUTPUT, NUM_HIDDEN))/NUM_HIDDEN**0.5) - 1./NUM_HIDDEN**0.5
b2 = 0.01 * np.ones(NUM_OUTPUT)

In [1650]:
# Concatenate all the weights and biases into one vector; this is necessary for check_grad
w = pack(W1, b1, W2, b2)
w.shape

(31810,)

In [1651]:
idxs = np.random.permutation(trainX.shape[0])[0:NUM_CHECK]
fCE_ = fCE((trainX[idxs, :]),(trainY[idxs]), w)
gradCE_ = gradCE((trainX[idxs, :]),(trainY[idxs]), w)

In [1652]:
fCE(trainX[idxs,:],trainY[idxs],w)

2.3033446922464607

In [1653]:
gradCE(trainX[idxs,:],trainY[idxs],w)[:100].shape

(100,)

In [1656]:
idxs = np.random.permutation(trainX.shape[0])[0:NUM_CHECK]

scipy.optimize.check_grad(
lambda w_:fCE(trainX[idxs,:],trainY[idxs],w_,),
lambda w_:gradCE(trainX[idxs,:],trainY[idxs],w_),
w)

13.465743116866799